In [1]:
%matplotlib inline

import os
import glob

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import keras
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, BatchNormalization, \
                         Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, LearningRateScheduler, \
                            ReduceLROnPlateau
import keras.backend as K

Using TensorFlow backend.


In [2]:
data_map = {}
max_class = 5
for i,f in enumerate(glob.glob("data/*.npy")):
    if i < max_class:
        data_map[os.path.basename(f)] = np.load(f)

train_samples_per_class = 2**10
test_samples_per_class = 2**8
train_data_map, test_data_map = {}, {}
for key in data_map:
    train_data_map[key] = data_map[key].reshape((-1, 28, 28))[:,:,:,np.newaxis][:train_samples_per_class]
    test_data_map[key] = data_map[key].reshape((-1, 28, 28))[:,:,:,np.newaxis][train_samples_per_class:train_samples_per_class+test_samples_per_class]
    train_data_map[key] = train_data_map[key].astype("float32") / 255
    test_data_map[key] = test_data_map[key].astype("float32") / 255

class_orders = data_map.keys()
train_data = np.concatenate([train_data_map[key] for key in class_orders])
test_data = np.concatenate([test_data_map[key] for key in class_orders])
train_target = [train_data_map[k].shape[0]*[i] for i,k in enumerate(class_orders)]
test_target = [test_data_map[k].shape[0]*[i] for i,k in enumerate(class_orders)]
train_target = np_utils.to_categorical(train_target)
test_target = np_utils.to_categorical(test_target)

In [3]:
print data_map.keys()
print train_data.shape, train_target.shape, test_data.shape, test_target.shape

['broccoli.npy', 'bench.npy', 'baseball bat.npy', 'matches.npy', 'hot air balloon.npy']
(5120, 28, 28, 1) (5120, 5) (1280, 28, 28, 1) (1280, 5)


In [4]:
num_class = len(data_map)

train_data_gen = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                                    height_shift_range=0.1, horizontal_flip=True,
                                    vertical_flip=False)
test_data_gen = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                                   height_shift_range=0.1, horizontal_flip=True,
                                   vertical_flip=False)
train_data_gen.fit(train_data)
test_data_gen.fit(test_data)

In [5]:
input_layer = Input(shape=(28, 28, 1), name='Input_layer')

# convblock 01
x = Conv2D(32, (3, 3), padding='same', activation='relu',
           input_shape=input_layer.shape, name='Conv01_layer')(input_layer)
x = Conv2D(32, (3, 3), activation='relu', name='Conv02_layer')(x)
#x = Conv2D(32, (3, 3), activation='relu', name='Conv02b_layer')(x)
x = MaxPooling2D(pool_size=(2, 2), name='MaxPool01_layer')(x)
#x = Dropout(0.9, name='Dropout01')

# convblock 02
x = Conv2D(32, (3, 3), padding='same', activation='relu',
           input_shape=input_layer.shape, name='Conv03_layer')(x)
x = Conv2D(32, (3, 3), activation='relu', name='Conv04_layer')(x)
x = Conv2D(32, (3, 3), activation='relu', name='Conv04b_layer')(x)
x = MaxPooling2D(pool_size=(2, 2), name='MaxPool02_layer')(x)
#x = Dropout(0.9, name='Dropout01')

# # convblock 03
# x = Conv2D(64, (3, 3), padding='same', activation='relu',
#            input_shape=input_layer.shape, name='Conv05_layer')(x)
# x = Conv2D(64, (3, 3), activation='relu', name='Conv06_layer')(x)
# x = MaxPooling2D(pool_size=(2, 2), name='MaxPool03_layer')(x)
# #x = Dropout(0.8, name='Dropout01')

# fully connected dense block
x = Flatten(name='Flatten_layer')(x)
x = Dense(512, activation='relu', name='Dense01_layer')(x)
x = BatchNormalization()(x)
x = Dense(num_class, name='logits_layer')(x)
#x = Dropout(0.5, name='Dropout01')
output = Activation('softmax', name='Softmax_layer')(x)

In [6]:
model = Model(input_layer, output)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_layer (InputLayer)     (None, 28, 28, 1)         0         
_________________________________________________________________
Conv01_layer (Conv2D)        (None, 28, 28, 32)        320       
_________________________________________________________________
Conv02_layer (Conv2D)        (None, 26, 26, 32)        9248      
_________________________________________________________________
MaxPool01_layer (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
Conv03_layer (Conv2D)        (None, 13, 13, 32)        9248      
_________________________________________________________________
Conv04_layer (Conv2D)        (None, 11, 11, 32)        9248      
_________________________________________________________________
Conv04b_layer (Conv2D)       (None, 9, 9, 32)          9248      
__________

In [ ]:
learning_rate = 0.1
decay = 0.1
batch_size = 256
epochs = 100

callbacks = [EarlyStopping(monitor='loss', min_delta=0.001, patience=10),
             LearningRateScheduler(lambda epoch:learning_rate/(1 + decay * epoch)),
             ReduceLROnPlateau(monitor='loss')]
             
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
K.set_value(model.optimizer.lr, learning_rate)
model.fit_generator(train_data_gen.flow(train_data, train_target, batch_size=batch_size),
                    validation_data=test_data_gen.flow(test_data, test_target, batch_size=batch_size),
                    steps_per_epoch=train_data.shape[0] // batch_size,
                    validation_steps=test_data.shape[0] // batch_size,
                    epochs=epochs,
                    workers=8,
                    callbacks=callbacks)

Epoch 1/100
20/20 [==============================] - 3s - loss: 3.8218 - acc: 0.4357 - val_loss: 9.8276 - val_acc: 0.3289
Epoch 2/100
20/20 [==============================] - 2s - loss: 1.4258 - acc: 0.5334 - val_loss: 5.3775 - val_acc: 0.3305
Epoch 3/100
20/20 [==============================] - 2s - loss: 1.0293 - acc: 0.6143 - val_loss: 5.9541 - val_acc: 0.3063
Epoch 4/100
20/20 [==============================] - 2s - loss: 0.9331 - acc: 0.6633 - val_loss: 1.3803 - val_acc: 0.5391
Epoch 5/100
20/20 [==============================] - 2s - loss: 0.8472 - acc: 0.6900 - val_loss: 1.4198 - val_acc: 0.5383
Epoch 6/100
20/20 [==============================] - 2s - loss: 0.7945 - acc: 0.7314 - val_loss: 1.7344 - val_acc: 0.4141
Epoch 7/100
20/20 [==============================] - 2s - loss: 0.7185 - acc: 0.7510 - val_loss: 1.4715 - val_acc: 0.4492
Epoch 8/100
20/20 [==============================] - 2s - loss: 0.7057 - acc: 0.7611 - val_loss: 1.2194 - val_acc: 0.4711
Epoch 9/100
20/20 [=====

20/20 [==============================] - 2s - loss: 0.2881 - acc: 0.9008 - val_loss: 0.3695 - val_acc: 0.8734
Epoch 68/100
20/20 [==============================] - 2s - loss: 0.3001 - acc: 0.8998 - val_loss: 0.3204 - val_acc: 0.8922
Epoch 69/100
20/20 [==============================] - 2s - loss: 0.2978 - acc: 0.8967 - val_loss: 0.3086 - val_acc: 0.8938
Epoch 70/100
20/20 [==============================] - 2s - loss: 0.2923 - acc: 0.8975 - val_loss: 0.3854 - val_acc: 0.8609
Epoch 71/100
20/20 [==============================] - 2s - loss: 0.2905 - acc: 0.8973 - val_loss: 0.3374 - val_acc: 0.8781
Epoch 72/100
20/20 [==============================] - 2s - loss: 0.2892 - acc: 0.8994 - val_loss: 0.3349 - val_acc: 0.8719
Epoch 73/100
20/20 [==============================] - 2s - loss: 0.2857 - acc: 0.8996 - val_loss: 0.3473 - val_acc: 0.8836
Epoch 74/100
20/20 [==============================] - 2s - loss: 0.2753 - acc: 0.9002 - val_loss: 0.3579 - val_acc: 0.8672
Epoch 75/100
20/20 [=========